In [ ]:
import tqdm.notebook
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)



inp = Input(shape=(320, 240, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
# neck_out = Dense(1, activation='linear', name='neck_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
# knee_out = Dense(1, activation='linear', name='knee_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
# head_out = Dense(1, activation='linear', name='head_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
#arm_out = Dense(1, activation='linear', name='arm_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)
# torso_out = Dense(1, activation='linear', name='torso_out')(dense)
# inner_leg_out = Dense(1, activation='linear', name='inner_leg_out')(dense)



model = Model(
    inputs=inp,
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        # neck_out,
        bicep_out,
        thigh_out,
        # knee_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        # head_out,
        wrist_out,
        #arm_out,
        shoulder_to_shoulder_out,
        # torso_out,
        # inner_leg_out
    ],
    name='conv_bodies'
)


###TRAINING VARIABLES

sample_count = 79999
epoch_count = 10
batch_size = 5
validation_count = 20000
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
dataset_name = "SuToBoCM"

folder_path = f"{dataset_name} - {sample_count}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")

model.summary()

from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [ ]:
class Database_Loader(keras.utils.Sequence):
    
    def __init__(self, image_location, data_location, sample_count, batch_size, dataset = "Surreact", shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", use_memory=False, load_data = True, random_sample = False):
        self.image_location = image_location
        self.data_location = data_location
        self.sample_count = sample_count
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.use_memory = use_memory
        self.load_data = load_data
        self.random_sample = random_sample
        self.dataset = dataset

        super().__init__(workers=2, use_multiprocessing=True)
        try:
            os.remove("./dataflowValid.txt")
        except:
            pass

        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
        else:
            self.IDs = [x for x in range(self.sample_count)]       
            self._load_data()

        super().__init__(workers=1, use_multiprocessing=False)
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(self.sample_count / self.batch_size))
    
    
    def _load_data(self):
        if self.load_data:
            self.data = dict()
            for index in self.IDs:
                self.data[index] = np.load(os.path.join(self.data_location + self.prefix + f"{index:06d}.npy"))

        if self.use_memory:
            self.images = dict()
            for index in self.IDs:
                self.images[index] = cv2.imread(os.path.join(self.image_location + self.prefix + f"{index:06d}.png"), cv2.IMREAD_GRAYSCALE)
    
    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 320, 240))
        y = {
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            # 'neck_out': [],
            'bicep_out': [],
            'thigh_out': [],
            # 'knee_out': [],
            'shoulder_to_wrist_out': [],
            'leg_out': [],
            'calf_out': [],
            # 'head_out': [],
            'wrist_out': [],
            # 'arm_out': [],
            'shoulder_to_shoulder_out': [],
            # 'torso_out': [],
            # 'inner_leg_out': [],
        }

        start_index = index * self.batch_size + 1
        with open(f"./dataflowValid.txt", "a") as validationFile:
            for i in range(self.batch_size):
                name = f"{self.IDs[(start_index + i)%self.sample_count]:06d}"
                validationFile.write("Sample Name: " + name +"\n")
                if self.load_data:
                    current_measurement = self.data[self.IDs[(start_index + i)%self.sample_count]][:-1]          
                else:
                    current_measurement = np.load(os.path.join(self.data_location + self.prefix + name + ".npy"))[:-1]
                    
                if self.use_memory:
                    X[i,] = self.images[self.IDs[(start_index + i)%self.sample_count]]
                    
                else:
                    X[i,] = cv2.imread(os.path.join(self.image_location + self.prefix + name + ".png"), cv2.IMREAD_GRAYSCALE)
                    
                validationFile.writelines([str(x)+"\n" for x in current_measurement])
                    
                if self.dataset == "Surreact":
                    y['chest_out'].append([current_measurement[0]])
                    y['waist_out'].append([current_measurement[1]])
                    y['pelvis_out'].append([current_measurement[2]])
                    # y['neck_out'].append([current_measurement[3]])
                    y['bicep_out'].append([current_measurement[4]])
                    y['thigh_out'].append([current_measurement[5]])
                    # y['knee_out'].append([current_measurement[6]])
                    y['shoulder_to_wrist_out'].append([current_measurement[7]])
                    y['leg_out'].append([current_measurement[8]])
                    y['calf_out'].append([current_measurement[9]])
                    # y['head_out'].append([current_measurement[10]])
                    y['wrist_out'].append([current_measurement[11]])
                    # y['arm_out'].append([current_measurement[12]])
                    y['shoulder_to_shoulder_out'].append([current_measurement[13]])
                    # y['torso_out'].append([current_measurement[14]])
                    # y['inner_leg_out'].append([current_measurement[15]])
                validationFile.write("===========\n")
            validationFile.write("END OF BATCH===========================================\n\n")

        for key, value in y.items():
            y[key] = np.array(value)
            
        return X, y
    
    def on_epoch_end(self):
        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
            self._load_data()

        if self.shuffle:
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(self.sample_count)
         

In [ ]:
quickTrain = {
                'image_location': "../Surreact-APose/train/imgs_nobg_frontEDITED/", 
                'data_location': "../Surreact-APose/train/measurementsCM/", 
                'sample_count': sample_count, 
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'use_memory': False,
                'random_sample': False,
                'shuffle': True}

quickValidate = {
                'image_location': "../Surreact-APose/test/imgs_nobg_frontEDITED/",
                'data_location': "../Surreact-APose/test/measurementsCM/",
                'sample_count': validation_count,
                'batch_size': batch_size,
                'seed': 69, #np.random.randint(0, 10000),
                'use_memory': False,
                'random_sample': False}

train_generator = Database_Loader(**quickTrain)
validation_generator = Database_Loader(**quickValidate)

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.losses import MeanSquaredError
from keras_tqdm import TQDMNotebookCallback


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        # 'neck_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        # 'knee_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        # 'head_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        # 'arm_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
        # 'torso_out': MeanSquaredError(),
        # 'inner_leg_out': MeanSquaredError()
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        # 'neck_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        # 'knee_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        # 'head_out': ['mae'],
        'wrist_out': ['mae'],
        # 'arm_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
        # 'torso_out': ['mae'],
        # 'inner_leg_out': ['mae'],
    },
)



checkpoint_filepath = './models/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=True,log_dir="./logs/"+folder_path),
        TQDMNotebookCallback(separator="", metric_format="{name}: {value:0.8f} || "),
        model_checkpoint_callback,
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=0
)

model.save("./models/full/model.keras")
# 
# for i in range(10000):
#     image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{6969+i:06d}.png", cv2.IMREAD_GRAYSCALE)
# 
#     print(model.predict(np.array([image])))
#     

In [ ]:
# from keras.models import load_model
# import cv2
# import numpy as np
# import pandas as pd
# import tqdm
# model = load_model("./models/full/SuToBo - 79999e10e20000v 1e0.97d.keras")
# 
# #ankle	arm-length	bicep	calf	chest	forearm	height	hip	leg-length	shoulder-breadth	shoulder-to-crotch	thigh	waist	wrist
# #0      1           2       3       4       5       6       7   8           9                   10                  11      12      13
# 
# def printResults(name, val1, val2):
#     if val2 == "N/A":
#         print(name, " - ", val1, val2)
#         return 
#     print(name, " - ", val1, val2, " error - ", abs(val1 - val2))
# 
# 
# 
# results_dataframe = pd.DataFrame(columns=["Chest", "Waist", "Pelvis", "Bicep", "Thigh", "Shoulder to wrist", "Leg", "Calf", "Wrist", "Shoulder to shoulder"])
# 
# 
# for i in tqdm.notebook.tqdm(range(8978)):
#     image = cv2.imread(f"../bodym-dataset/front/images/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
#     data = np.load(f"../bodym-dataset/front/measurements/Avatar_{i:06d}.npy")
#     predictions = model.predict(np.array([image]))
#     series = pd.DataFrame({
#         "Chest": abs(predictions[0][0] - data[4]), 
#         "Waist": abs(predictions[1][0] - data[12]), 
#         "Pelvis": abs(predictions[2][0] - data[7]), 
#         "Bicep": abs(predictions[3][0] - data[2]), 
#         "Thigh": abs(predictions[4][0] - data[11]), 
#         "Shoulder to wrist": abs(predictions[5][0] - data[1]), 
#         "Leg": abs(predictions[6][0] - data[8]),
#         "Calf": abs(predictions[7][0] - data[3]), 
#         "Wrist": abs(predictions[8][0] - data[13]),
#         "Shoulder to shoulder": abs(predictions[10][0] - data[9])}, index=[0])
#     results_dataframe = pd.concat([results_dataframe, series], ignore_index=True)
#     # printResults("Chest", predictions[0], data[4])
#     # printResults("Waist", predictions[1], data[12])
#     # printResults("Pelvis", predictions[2], data[7])
#     # printResults("Bicep", predictions[3], data[2])
#     # printResults("Thigh", predictions[4], data[11])
#     # printResults("Shoulder to wrist", predictions[5], data[1])
#     # printResults("Calf", predictions[6], data[3])
#     # printResults("Wrist", predictions[7], data[13])
#     # printResults("Shoulder to shoulder", predictions[8], data[9])
#     # printResults("Inner leg", predictions[9], data[8])
# 
# # display(results_dataframe)
# results_dataframe.to_csv("SuToBoCM - real.csv", index=False)

In [31]:
# print(results_dataframe.mean())

Chest                   12.358825
Waist                    9.884277
Pelvis                   9.241092
Bicep                    3.561877
Thigh                    4.855555
Shoulder to wrist        4.259165
Leg                      5.677104
Calf                     4.770560
Wrist                    1.708310
Shoulder to shoulder     3.121616
dtype: float32


In [28]:
# from keras.models import load_model
# import cv2
# import numpy as np
# import pandas as pd
# import tqdm
# model = load_model("./models/full/SuToBo - 79999e10e20000v 1e0.97d.keras")
# 
# 
# results_dataframe_synthetic = pd.DataFrame(columns=["Chest", "Waist", "Pelvis", "Bicep", "Thigh", "Shoulder to wrist", "Leg", "Calf", "Wrist", "Shoulder to shoulder"])
# 
# 
# for i in tqdm.notebook.tqdm(range(20000)):
#     image = cv2.imread(f"../Surreact-APose/test/images_front/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
#     data = np.load(f"../Surreact-APose/test/measurementsCM/Avatar_{i:06d}.npy")
#     predictions = model.predict(np.array([image]))
#     series = pd.DataFrame({
#         "Chest": abs(predictions[0][0] - data[0]),
#         "Waist": abs(predictions[1][0] - data[1]),
#         "Pelvis": abs(predictions[2][0] - data[2]),
#         "Bicep": abs(predictions[3][0] - data[4]),
#         "Thigh": abs(predictions[4][0] - data[5]),
#         "Shoulder to wrist": abs(predictions[5][0] - data[7]),
#         "Leg": abs(predictions[6][0] - data[8]),
#         "Calf": abs(predictions[7][0] - data[9]),
#         "Wrist": abs(predictions[8][0] - data[11]),
#         "Shoulder to shoulder": abs(predictions[10][0] - data[13])}, index=[0])
#     results_dataframe_synthetic = pd.concat([results_dataframe_synthetic, series], ignore_index=True)
# 
# # display(results_dataframe)
# results_dataframe_synthetic.to_csv("SuToBoCM - synthetic2.csv", index=False)

  0%|          | 0/20000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


/tmp/ipykernel_321075/1455682991.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_dataframe_synthetic = pd.concat([results_dataframe_synthetic, series], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [29]:
# print(results_dataframe_synthetic.mean())

Chest                   10.847208
Waist                   13.801395
Pelvis                   8.776840
Bicep                    4.028211
Thigh                    5.041528
Shoulder to wrist        3.661919
Leg                      5.556999
Calf                     3.340129
Wrist                    1.592633
Shoulder to shoulder     4.180833
dtype: float32


In [30]:
# # Averages
# import pandas
# 
# measures = train_generator.data
# col = ["chest", "waist", "pelvis", "neck", "bicep","thigh", "knee", "arm", "leg", "calf", "head", "wrist", "armSpan", "shoulder", "torso","iLeg", "height"]
# df = pandas.DataFrame(measures.values(), columns=col)
# for i in col:
#     avg = df.loc[:, i].mean()
#     result = results_dataframe.mean()
#     print(i, df.loc[:, i].mean(), )

chest 94.23777471214385
waist 80.07869769435285
pelvis 100.54852472539844
neck 37.28966737527179
bicep 29.353643487324533
thigh 51.31717446677254
knee 37.72483493665183
arm 55.02483319348413
leg 83.97085905644197
calf 43.98375918857878
head 57.101469326633556
wrist 17.806347924152533
armSpan 188.9231
shoulder 38.86983096599579
torso 52.05414593219757
iLeg 77.94546840896606
height 181.38332837195924


In [ ]:
# #Data modifiaction
# 
# import os
# import tqdm
# import numpy as np
# 
# avatar_count = 20000
# counter = 0
# 
# for number in tqdm.tqdm(range(avatar_count)):
#     # if number == 5:
#     #     continue
#     file = (tuple(np.load(f"../Surreact-APose/test/bodymeasurements/Avatar_{number:06d}.npy", allow_pickle=True)))
#     measurements = []
#     for value in file:
#         measurements.append(float(value * 100))
#     np.save(f"../Surreact-APose/test/measurementsCM/Avatar_{counter:06d}.npy", measurements)
#     counter += 1

In [ ]:
# # Image modification
# 
# import os
# import cv2
# import numpy as np
# import tqdm
# 
# 
# avatar_count = 20000
# counter = 0
# 
# for number in tqdm.notebook.tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     image = cv2.imread(f"../Surreact-APose/test/imgs_nobg_front/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     alpha_channel = image[:, :, 3]
#     body_indices = alpha_channel > 30
#     image[body_indices] = [255, 255, 255, 255]
#     indices = np.where(body_indices)
#     leftmost_pixel = np.min(indices[1])
#     rightmost_pixel = np.max(indices[1])
#     topmost_pixel = max(np.min(indices[0]), 0)
#     bottommost_pixel = min(np.max(indices[0]), 239)
#     cropped = image[topmost_pixel:bottommost_pixel, leftmost_pixel:rightmost_pixel]
# 
#     border_height = 320 - cropped.shape[0]
#     border_width = 240 - cropped.shape[1]
# 
#     # Calculate the border sizes
#     top_border = border_height // 2
#     bottom_border = border_height - top_border
#     left_border = border_width // 2
#     right_border = border_width - left_border
# 
#     # Add the border
#     bordered_image = cv2.copyMakeBorder(cropped, top_border, bottom_border, left_border, right_border, cv2.BORDER_CONSTANT, value=(0,0,0,0))
#     bordered_image = cv2.cvtColor(bordered_image, cv2.COLOR_RGBA2GRAY)
# 
# 
#     cv2.imwrite(f"./Export/Surreact-APose/test/images_front/Avatar_{counter:06d}.png", bordered_image)
#     counter +=1

In [ ]:
# Image validation

# import os
# import cv2
# import numpy as np
# from tqdm import tqdm
# 
# 
# avatar_count = 79998
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     if image.shape != (240, 320, 4):
#         print(image.shape, number)
#     counter +=1

In [ ]:
# #FOR BODY_M DATASET
# import csv
# import numpy as np
# import os
# 
# import cv2
# import tqdm.notebook
# 
# counter = 0
# for folder in tqdm.notebook.tqdm(["train","testA","testB"]):
# 
#     with open('../bodym-dataset/'+folder+'/subject_to_photo_map.csv', 'r') as f:
#         reader = csv.reader(f)
#         uuid_raw = list(reader)
#     from collections import defaultdict
# 
#     uuid_map = defaultdict(list)
# 
#     for subject_id, photo_id in uuid_raw[1:]:
#         uuid_map[subject_id].append(photo_id)
#     image_data = {}
# 
#     with open('../bodym-dataset/'+folder+'/measurements.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
# 
# 
#             for image_id in uuid_map[subject_id]:
#                 image_data[image_id] = np.array(list(map(float, line[1:])))
# 
# 
#     for subject_id, images in tqdm.notebook.tqdm(uuid_map.items()):
#         for image in images:
#             measurements = np.array(image_data[image])
#             silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#             silhouette = cv2.resize(silhouette,(240, 320)) #may be wrong
#             cv2.imwrite(os.path.join(f'../bodym-dataset/front/images/Avatar_{counter:06d}.png'), silhouette)
#             np.save(os.path.join(f'../bodym-dataset/front/measurements/Avatar_{counter:06d}.npy'), measurements)
#             counter += 1
